In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [15]:
df = pd.read_csv('input/train.csv')
# df = pd.read_csv('input/train_min.csv')  # small data

fix_data_skew = True

if fix_data_skew:
    trues = df.loc[df['target'] == 1]
    falses = df.loc[df['target'] != 1].sample(frac=1)[:len(trues)]
    data = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    data = df
    
data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
4458,train_44958,1,7.1097,-3.6626,12.9916,5.4532,9.1796,-10.4245,3.3715,13.7593,...,6.0756,14.3647,0.9228,2.4267,17.6248,-0.2987,10.4272,8.7023,15.5484,-2.7033
19707,train_196453,1,8.3518,-6.5567,14.6280,7.3526,12.1076,-7.3077,4.6128,10.7624,...,4.1633,5.9991,2.7654,-3.5509,16.6949,0.8259,9.9904,8.1459,12.3860,-21.0266
3702,train_37314,1,17.7336,2.4519,11.0154,11.1458,10.8296,-14.1031,4.8839,17.7223,...,0.5902,9.5091,0.8411,0.8898,15.6398,1.8360,0.3746,8.3489,14.9042,-12.4719
5067,train_51011,1,17.6923,-4.7484,8.4909,7.8138,11.0619,13.4346,5.2511,17.1373,...,2.6474,6.8746,-0.0815,7.8560,21.9572,-1.9430,6.1172,7.9742,18.4485,-8.7427
11323,train_113002,1,9.0074,4.7071,12.3721,5.3604,11.3085,-11.3558,5.7844,12.6466,...,-0.8608,5.7127,2.2585,6.7968,22.2517,0.8920,0.5512,9.5950,15.6518,7.5509


In [16]:
X, y = data.iloc[:,2:].values, data.iloc[:,1].values

# std scaling
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [40]:
#cv: 0.8597059678850029
#    0.8584394316060081 (with data skew fix)
lg = SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)

#cv: 0.8543658603036868
#    0.8466847795424428 (with data skew fix)
mlp = MLPClassifier(solver='lbfgs', alpha=0.001,
                    hidden_layer_sizes=(5, 2), random_state=1)

# cv: 0.83 (with data skew fix)
rf = RandomForestClassifier(n_estimators=100, criterion='entropy')

#cv: 0.8883828469044662
#    0.8874777137937799 (with data skew fix)
gnb = GaussianNB()

#cv: 0.8608604117713252 (with data skew fix)
lda = LinearDiscriminantAnalysis()

#cv: 0.820027075184469 (with data skew fix)
qda = QuadraticDiscriminantAnalysis(tol=1e-12)

# cv: 0.8616254422551686 (with data skew fix)
ada = AdaBoostClassifier(lg)

# cv: 0.8614224931614438 (with data skew fix)
bg = BaggingClassifier(lg);

cls = bg

# try grid-search
params = { 'base_estimator': [ None, lg ] }
cls = GridSearchCV(estimator=cls, param_grid=params, cv=5, scoring='roc_auc')
cls.fit(X_scaled, y)
print(cls.best_params_)
print(cls.best_score_)

# single
#cv_scores = cross_val_score(cls, X_scaled, y, cv=5, scoring='roc_auc')
#print(cv_scores)
#print(np.average(cv_scores))

# voting
#estimators = [ ('lda', lda), ('gnb', gnb), ('lg', lg), ('mlp', mlp) ]

#vc = VotingClassifier(estimators=estimators.copy(), voting='soft')
#estimators.append(('vc', vc))

#for label, cls in estimators:
#   scores = cross_val_score(cls, X_scaled, y, cv=5, scoring='roc_auc')
#   print("Score: %0.16f (+/- %0.16f) [%s]" % (scores.mean(), scores.std(), label))

{'base_estimator': SGDClassifier(alpha=0.3, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=1e-07,
       validation_fraction=0.1, verbose=0, warm_start=False)}
0.8614224931614438


In [18]:
cls.fit(X_scaled, y)

VotingClassifier(estimators=[('gnb', GaussianNB(priors=None, var_smoothing=1e-09)), ('lg', SGDClassifier(alpha=0.3, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5000,
       n_iter=None, n...True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [19]:
# create final output
test = pd.read_csv('input/test.csv')
test_ids, test_x = test.iloc[:,0], test.iloc[:,1:]

test_x_scaled = scaler.transform(test_x)
test_y = cls.predict_proba(test_x_scaled)

In [20]:
# output the result
sub_ids = pd.DataFrame(test_ids)
sub_y = pd.DataFrame(pd.DataFrame(test_y).iloc[:,1])

sub = pd.DataFrame(np.hstack((sub_ids, sub_y)))
sub.to_csv('output.csv', header=['ID_code', 'target'], index=False)

output_check = pd.read_csv('output.csv')
output_check.head()

,ID_code,target
0,test_0,0.645082
1,test_1,0.660114
2,test_2,0.506549
3,test_3,0.620484
4,test_4,0.322633
